<a href="https://colab.research.google.com/github/curiouswalk/manim/blob/main/source/wave_surface_3d/wave_surface_3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CuriousWalk
#### [`www.curiouswalk.com`](https://www.curiouswalk.com)
#### [`link.curiouswalk.com/manim`](https://link.curiouswalk.com/manim)

# Manim

Manim is an engine for precise programmatic animations, designed for creating explanatory math videos.

> The Manim Community Developers. (2024). Manim – Mathematical Animation Framework (Version v0.18.0) [Computer software].  [www.manim.community](https://www.manim.community/)

# Setup

[Installation Guide](https://docs.manim.community/en/stable/installation.html)

Run this cell to get started.

In [ ]:
from IPython.display import clear_output

!sudo apt update

!sudo apt install libcairo2-dev libpango1.0-dev ffmpeg

# # LaTeX installation
# !sudo apt install texlive texlive-latex-extra

# # LaTeX additional packages
# !sudo apt install texlive-science texlive-fonts-extra

!pip install manim

clear_output() # clears cell output

!manim --version

exit() # restarts runtime

# Imports

In [ ]:
from manim import *
config.disable_caching = True

# Example

```python
%%manim -ql NewScene
class NewScene(Scene):
    def construct(self):
      ...  
```

The first line of code, `%%manim -ql NewScene` is a magic command, it only works within Jupyter notebooks. The flag `-ql` specifies low render quality (480p, 15fps), followed by the name of the scene class.

The other options for render quality are `-qm`, `-qh`, and `-qk` for medium (720p, 30fps), high (1080p, 60fps), and 4k (2160p, 60fps) quality, respectively.

This animation expands Manim's logo into its banner.  [[source]](https://docs.manim.community/en/stable/reference/manim.mobject.logo.ManimBanner.html)

In [ ]:
%%manim -ql DarkThemeBanner

class DarkThemeBanner(Scene):
    def construct(self):
        banner = ManimBanner()
        self.play(banner.create())
        self.play(banner.expand())
        self.wait()

# Wave Animation
### 3D Manim Scene

## RadialWave

The `RadialWave` class is taken from `manim-physics` plugin. [[source]](https://manim-physics.readthedocs.io/en/latest/_modules/manim_physics/wave.html#RadialWave)

>**Manim Physics** is a 2D physics simulation plugin for Manim that allows for making complicated scenes in various branches of Physics such as rigid mechanics, electromagnetism, wave, etc.
>
>[https://manim-physics.readthedocs.io/en/latest/](https://manim-physics.readthedocs.io/en/latest/)

In [ ]:
class RadialWave(Surface):
    def __init__(
        self,
        *sources, # sources of disturbance (locations)
        wavelength: float = 1,
        period: float = 1,
        amplitude: float = 0.1,
        x_range = [-5, 5],
        y_range = [-5, 5],
        **kwargs,
    ) -> None:

        self.wavelength = wavelength
        self.period = period
        self.amplitude = amplitude
        self.time = 0
        self.kwargs = kwargs
        self.sources = sources

        super().__init__(
            lambda u, v: np.array([u, v, self._wave_z(u, v, sources)]),
            u_range=x_range,
            v_range=y_range,
            **kwargs,
        )

    def _wave_z(self, u: float, v: float, sources) -> float:
        z = 0
        for source in sources:
            x0, y0, _ = source
            z += self.amplitude * np.sin(
                (2 * PI / self.wavelength) * ((u - x0) ** 2 + (v - y0) ** 2) ** 0.5
                - 2 * PI * self.time / self.period
            )
        return z

    def _update_wave(self, mob: Mobject, dt: float) -> None:
        self.time += dt
        mob.match_points(
            Surface(
                lambda u, v: np.array([u, v, self._wave_z(u, v, self.sources)]),
                u_range=self.u_range,
                v_range=self.v_range,
                **self.kwargs,
            )
        )

    def start_wave(self):
        """Animate the wave propagation."""
        self.add_updater(self._update_wave)

    def stop_wave(self):
        """Stop animating the wave propagation."""
        self.remove_updater(self._update_wave)



# Scene 1
Wave Transformation

In [ ]:
%%manim -ql WaveTransform

class WaveTransform(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(30 * DEGREES, 45 * DEGREES, zoom=2)

        surface = RadialWave(
            checkerboard_colors= ["#107ab0"],
            stroke_width=2,
            stroke_color= "#280137")

        wave = RadialWave(
            ORIGIN,
            period=2,
            amplitude=.2,
            checkerboard_colors= ["#107ab0"],
            stroke_width=2,
            stroke_color= "#280137")

        self.add(surface)

        self.wait()

        self.play(ReplacementTransform(surface, wave))
        wave.start_wave()

        self.wait(4)
        wave.stop_wave()


# Scene 2

Color Change & Camera Rotation

In [ ]:
%%manim -ql ColorChangeImg

class ColorChangeImg(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 0 * DEGREES)

        wave = RadialWave(
            ORIGIN,
            period=2,
            checkerboard_colors=[BLACK, WHITE],
            stroke_width=2,
            stroke_color=BLACK,
        )

        self.add(wave)

In [ ]:
%%manim -ql ColorChange

class ColorChange(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 0 * DEGREES)

        wave = RadialWave(
            ORIGIN,
            period=2,
            checkerboard_colors=["#107ab0"],
            stroke_width=2,
            stroke_color="#280137",
        )

        self.add(wave)

        self.wait(1.5)

        self.play(
            wave.animate(run_time=1.5)
            .set_fill_by_checkerboard(BLACK)
            .set_stroke(WHITE)
        )

        self.wait(1.5)

        self.play(
            wave.animate(run_time=1.5)
            .set_fill_by_checkerboard(BLACK, WHITE)
            .set_stroke(BLACK)
        )

        self.wait(1.5)

        self.play(
            wave.animate(run_time=1.5).set_fill_by_checkerboard(
                "#8d2653", "#538d26", "#26538d"
            )
        )

        self.wait(1.5)


In [ ]:
%%manim -ql ColorChangeAndRotate

class ColorChangeAndRotate(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 0 * DEGREES)

        wave = RadialWave(
            ORIGIN,
            period=2,
            checkerboard_colors=[BLACK],
            stroke_width=2,
            stroke_color=WHITE,
        )

        self.add(wave)

        wave.start_wave()

        self.play(
            self.camera.theta_tracker.animate(rate_func=linear, run_time=2).set_value(
                PI * 0.25
            )
        )

        self.play(
            wave.animate.set_fill_by_checkerboard(BLACK, WHITE).set_stroke(BLACK),
            self.camera.theta_tracker.animate(rate_func=linear, run_time=2).set_value(
                PI * 0.5
            ),
        )

        self.play(
            wave.animate.set_fill_by_checkerboard("#8d2653", "#538d26", "#26538d"),
            self.camera.theta_tracker.animate(rate_func=linear, run_time=2).set_value(
                PI * 0.75
            ),
        )

        self.play(
            self.camera.theta_tracker.animate(rate_func=linear, run_time=2).set_value(
                PI
            )
        )

        wave.stop_wave()


# Scene 3
Single Source of Disturbance

In [ ]:
%%manim -ql SingleSourceImg

class SingleSourceImg(ThreeDScene):
    def construct(self):

        points = (UL, UR, DL, DR)  # source location

        circle = Circle(0.75, color="#ff073a", fill_opacity=0.5)

        waves = VGroup()

        for i in points:

            pos = i * 2.5

            waves.add(
                RadialWave(
                    pos,
                    checkerboard_colors=["#107ab0"],
                    stroke_width=1,
                    stroke_color="#280137",
                ).add(circle.copy().move_to(pos))
            )

        waves.arrange_in_grid(2, 2)

        self.set_camera_orientation(zoom=config.frame_height / waves.height)

        self.add(waves)


In [ ]:
%%manim -ql OneSource

class OneSource(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 45 * DEGREES)

        point = UR * 2.5  # source location

        circle = Circle(0.75, color="#ff073a", fill_opacity=0.5).move_to(point)

        wave = RadialWave(
            point,
            period=2,
            checkerboard_colors=["#107ab0"],
            stroke_width=2,
            stroke_color="#280137",
        )

        self.add(circle, wave)

        wave.start_wave()

        self.play(FadeOut(circle, scale=0))

        self.wait(3)

        wave.stop_wave()


# Scene 4
Multiple Sources of Disturbance

In [ ]:
%%manim -ql TwoSourceImg

class TwoSourceImg(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 45 * DEGREES)

        points = (UR * 2 + RIGHT * 1.5, UR * 2 + UP * 1.5)
        colors = ("#0cff0c", "#ff073a")

        circles = VGroup()

        for p, c in zip(points, colors):
            circles.add(Circle(0.75, color=c, fill_opacity=0.5).move_to(p))

        wave = RadialWave(
            points[0],
            points[1],
            period=2,
            checkerboard_colors=["#107ab0"],
            stroke_width=2,
            stroke_color="#280137",
        )

        self.add(wave, circles)


In [ ]:
%%manim -ql TwoSource

class TwoSource(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 45 * DEGREES)

        points = (UR * 2 + RIGHT * 1.5, UR * 2 + UP * 1.5)
        colors = ("#0cff0c", "#ff073a")

        circles = VGroup()

        for p, c in zip(points, colors):
            circles.add(Circle(0.75, color=c, fill_opacity=0.5).move_to(p))

        wave = RadialWave(
            points[0],
            points[1],
            period=2,
            checkerboard_colors=["#107ab0"],
            stroke_width=2,
            stroke_color="#280137",
        )

        self.add(wave, circles)

        self.wait()

        wave.start_wave()

        self.play(*[FadeOut(cir, scale=0) for cir in circles])

        self.wait(3)

        wave.stop_wave()


In [ ]:
%%manim -ql ThreeSourceImg

class ThreeSourceImg(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 45 * DEGREES)

        points = (DL * 2, DOWN * 1.5, LEFT * 1.5)

        colors = ("#ff073a", "#0cff0c", "#fd3c06")

        circles = VGroup()

        for p, c in zip(points, colors):
            circles.add(Circle(0.75, color=c, fill_opacity=0.5).move_to(p))

        wave = RadialWave(
            points[0],
            points[1],
            period=2,
            checkerboard_colors=["#107ab0"],
            stroke_width=2,
            stroke_color="#280137",
        )

        self.add(wave, circles)


In [ ]:
%%manim -ql ThreeSource

class ThreeSource(ThreeDScene):
    def construct(self):

        self.set_camera_orientation(60 * DEGREES, 45 * DEGREES)

        points = (DL * 2, DOWN * 1.5, LEFT * 1.5)
        colors = ("#ff073a", "#0cff0c", "#fd3c06")

        circles = VGroup()

        for p, c in zip(points, colors):
            circles.add(Circle(0.75, color=c, fill_opacity=0.5).move_to(p))

        wave = RadialWave(
            points[0],
            points[1],
            period=2,
            checkerboard_colors=["#107ab0"],
            stroke_width=2,
            stroke_color="#280137",
        )

        self.add(wave, circles)

        self.wait()

        wave.start_wave()

        self.play(*[FadeOut(cir, scale=0) for cir in circles])

        self.wait(3)

        wave.stop_wave()
